<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


Build an image classifier using the VGG16 pre-trained model, evaluate it and compare its performance to ResNet50

<a id="item41"></a>

## Download Data

In [1]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2020-03-07 19:53:55--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  27.5MB/s    in 9.0s    

2020-03-07 19:54:05 (27.7 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [ ]:
!unzip concrete_data_week4.zip

<a id="item42"></a>

## Part 1

1. Import libraries, modules and packages
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. 
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.


In [0]:
num_classes = 2

image_resize = 224

batch_size_training = 50 # I'm facing OOM(Out of Memory Error) when training VGG16, so I have to reduce the batch size.
batch_size_validation = 100


In [0]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [4]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [5]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [6]:
model = Sequential()

In [7]:
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

In [0]:
model.add(Dense(num_classes, activation='softmax'))

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 14,715,714
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.layers[0].trainable = False

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [13]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)
# I'm facing OOM(Out of Memory Error) when training VGG16, so I have to reduce the batch size.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/2
601/601 [==============================] - 252s 419ms/step - loss: 0.1066 - acc: 0.9634 - val_loss: 0.0209 - val_acc: 0.9952
Epoch 2/2
601/601 [==============================] - 246s 409ms/step - loss: 0.0160 - acc: 0.9961 - val_loss: 0.0131 - val_acc: 0.9974


In [0]:
model.save('classifier_vgg16_model.h5')

<a id="item43"></a>

## Part 2

1. Load the saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set.
3. Use the **evaluate_generator** method to evaluate models on the test data.
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.

In [15]:
from keras.models import load_model
model_ResNet = load_model('classifier_resnet_model.h5')
model_VGG16 = load_model('classifier_vgg16_model.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:350: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [16]:
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle = False)

Found 500 images belonging to 2 classes.


In [0]:
steps_per_epoch_test = len(test_generator)

In [18]:
model_VGG16.evaluate_generator(test_generator, steps=steps_per_epoch_test)

[0.011108655223622917, 0.998]

In [19]:
model_ResNet.evaluate_generator(test_generator, steps=steps_per_epoch_test)

[0.1863269196562469, 0.9239999990463257]

<a id="item44"></a>

## Part 3

1. Use the **predict_generator** method to predict the class of the images in the test data.
2. Report the class predictions of the first five images.

In [28]:
for i in range(0,5):
  if model_VGG16.predict_generator(test_generator, steps=steps_per_epoch_test)[i][1] <= 0.5:
    print('Negative')
  else:
    print('Positive')

Negative
Negative
Negative
Negative
Negative


In [29]:
for i in range(0,5):
  if model_ResNet.predict_generator(test_generator, steps=steps_per_epoch_test)[i][1] <= 0.5:
    print('Negative')
  else:
    print('Positive')

Negative
Negative
Negative
Negative
Negative


<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).